In [1]:
import os
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pickle

In [2]:
PATH = os.getcwd()
PATH_DATA = PATH + '/data'
PATH_DATA_en_BDNEWS24 = PATH + '/data/en_BDNEWS24'

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/avijitmandal/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/avijitmandal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/avijitmandal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
stop_words = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def rem_stop_words(text):
    tokens = word_tokenize(text)
    tokens_filtered= [word for word in tokens if not word in stop_words]
    return (" ").join(tokens_filtered)

def rem_punctions(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(text)
    return (" ").join(tokens)

def lemma(text):
    tokens=word_tokenize(text)
    afterlemmatize = [lemmatizer.lemmatize(word.lower()) for word in tokens ]
    return (" ").join(afterlemmatize)

In [5]:
def modify_list(posting_list):
    n = len(posting_list)
    count = 1
    res = []
    if n == 0:
        return res
    for i in range(1,n):
        if posting_list[i] == posting_list[i-1]:
            count += 1
        else:
            res.append((posting_list[i-1], count))
            count = 1
            
    res.append((posting_list[n-1], count))
    
    return res

In [11]:
def get_inverted_index():
    
    os.chdir(PATH_DATA_en_BDNEWS24)

    inverted_index = {}
    for dirname in os.listdir(os.getcwd()):
        
        for filename in os.listdir(os.path.join(os.getcwd(),dirname)):
            with open(os.path.join(os.path.join(os.getcwd(), dirname),filename), 'r') as f:
                s = f.read()
                s = s.split('<TEXT>')[1].split('</TEXT>')[0]
                
                s = rem_stop_words(s)
                s = rem_punctions(s)
                s = lemma(s)
                
                tokens=word_tokenize(s)
                for token in tokens:
                    if token in inverted_index.keys():
                        inverted_index[token].append(filename)
                    else:
                        inverted_index[token]=[filename]

    for key in inverted_index.keys():
        inverted_index[key].sort()
        inverted_index[key] = modify_list(inverted_index[key])
        
    return inverted_index

In [12]:
inverted_index = get_inverted_index()

In [13]:
os.chdir(PATH)
dbfile = open('model_queries_11.pth', 'wb')
pickle.dump(inverted_index, dbfile)                     
dbfile.close()

In [26]:
os.chdir(PATH)
dbfile = open('model_queries_11.pth', 'rb')     
newdict = pickle.load(dbfile)
dbfile.close

<function BufferedReader.close>

In [27]:
from bs4 import BeautifulSoup
os.chdir(PATH_DATA)

with open("raw_query.txt", "rb") as f:
    data = f.read().decode("UTF-8")

In [28]:
def parse(title):
    title=rem_stop_words(title)
    title=rem_punctions(title)
    title=lemma(title)
    filtered_list = title.split(" ")
    return filtered_list

In [29]:
soup = BeautifulSoup(data, 'html.parser')
query_dic = {}
for topic in soup.topics:
    try:
        title = topic.title.text
        query_dic[(topic.num.text)] = parse(title)
    except:
        pass

In [30]:
with open("queries_13.txt", "w") as text_file:
    for key in query_dic:
        res = " ".join(str(x) for x in query_dic[key])
        text_file.write(key + "," + res + "\n")

In [31]:
## 1C

In [32]:
with open("queries_13.txt", "rb") as f:
    query_data = f.read().decode("UTF-8")

In [33]:
query_ = query_data.split("\n")

In [34]:
parsed_query_dic = {}
for line in query_:
    query_full = line.split(",")
    if len(query_full) > 1:
        try:
            diff_words = query_full[1].split(" ")
        except:
            pass
        parsed_query_dic[query_full[0]] = diff_words

In [35]:
def merge_list(list1, list2):
        
    n, m = len(list1), len(list2)
    i, j = 0, 0
    res = []

    while i < n and j < m:
        if list1[i] == list2[j]:
            res.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1
    return res

In [36]:
def get_list(token, newdict):
    curr = []
    try:
        curr = newdict[token]
    except:
        pass
    res = []
    for val in curr:
        res.append(val[0])
    return res

In [37]:
def get_results(parsed_query, inverted_index):
    
    '''
    INPUT: parsed_query_dic: dictionaries having quiries
           inverted_index: retrivied from pth file
    OUTPUT: result: dictionary containing result
    '''
    result = {}
    for key in parsed_query.keys():
        query = parsed_query[key]
        tokens = []

        for token in query:
            curr = get_list(token, inverted_index)
            tokens.append((len(curr), token))
        tokens.sort()
        res = get_list(tokens[0][1], inverted_index)

        for token_ in tokens:
            token = token_[1]
            curr = get_list(token, inverted_index)
            res = merge_list(res, curr)
        result[key] = res
    return result

In [38]:
result_dic = get_results(parsed_query_dic, newdict)

In [39]:
with open("PAT1_13_results.txt", "w") as text_file:
    for key in result_dic:
        res = " ".join(str(x) for x in result_dic[key])
        text_file.write(key + ":" + res + "\n")